In [3]:
import os
import re
import cv2
import sys
import glob
import json
import math
import pickle
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
from pathlib import Path 
import matplotlib.pyplot as plt
from collections import defaultdict
from multiprocessing import Pool

In [7]:
def process(img_path):
    img = cv2.imread(str(img_path))
    # img path = a/b/c/d.jpg, convert to a/b/c_d.jpg
    cv2.imwrite(str(_saved_dir / '_'.join(img_path.parts[-2:])), img)
    
_root_img_dir = Path('/home/ptthang/d2.cattle/Ground_truth_aligned/aligned_images')
_all_img_paths = list(_root_img_dir.glob('**/*.jpg'))
# i will train all three of this, including the test set, but aligned, then infer on the original test set
_saved_dir = Path('/home/ptthang/d2.cattle/Ground_truth_aligned/train_aligned_imgs')
_saved_dir.mkdir(exist_ok=True)
with Pool(8) as p:
    for _ in tqdm(p.imap_unordered(process, _all_img_paths), total=len(_all_img_paths)):
        pass



 68%|██████▊   | 357/524 [00:00<00:00, 3565.37it/s]

100%|██████████| 524/524 [00:00<00:00, 3555.71it/s]


In [17]:
import itertools
_root_json_dir = Path('/home/ptthang/d2.cattle/Ground_truth_aligned/')
_all_json_files = list(_root_json_dir.glob('*.json'))
input_jsons = {}
for json_file in _all_json_files:
    with open(json_file, 'r') as f:
        input_jsons[json_file.stem] = json.load(f)

# [[], [], []] to []
output_data = {
    "images": [],
    "annotations": []
}
id = 1
for prefix, input_json in input_jsons.items():
    prefix = prefix.replace('_aligned', '')
    for item in tqdm(input_json):
        for image_id, image_data in item.items():
            image = Image.open(f'{_saved_dir}/{prefix}_{image_id}.jpg')
            image_width, image_height = image.size
            image = {
                "license": None,
                "file_name": f"{prefix}_{image_id}.jpg",
                "coco_url": None,
                "height": image_height,
                "width": image_width,
                "id": id
            }
            output_data["images"].append(image)

            annotation = {
                "segmentation": None,
                "num_keypoints": len(image_data["points"]) // 3,
                "area": (image_data["box"][2]) * (image_data["box"][3]),
                "iscrowd": 0,
                "keypoints": image_data["points"],
                "image_id": id,
                "bbox": image_data["box"],
                "category_id": 1,
                "id": id,
                "inmodal_bbox": None,
                "inmodal_seg": None
            }
            output_data["annotations"].append(annotation)
            id += 1

end_data = {
    "categories": [
        {
            "supercategory": "cow",
            "id": 1,
            "name": "cattle",
            "keypoints": [
                "p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8", "p9", "p10", "p11", "p12", "p13"
            ],
            "skeleton": [
                [1, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7], [6, 8], [7, 9], [8, 10], [9, 11], [10, 12], [11, 13], [12, 13]
            ]
        }
    ]
}
output_data.update(end_data)

output_json = json.dumps(output_data, indent=4)

with open(_saved_dir/'../aligned_train.json', 'w') as outfile:
    json.dump(output_data, outfile, indent=4)

100%|██████████| 204/204 [00:00<00:00, 36943.05it/s]


In [18]:
!ln -sf /home/ptthang/d2.cattle/Ground_truth_aligned/train_aligned_imgs /home/ptthang/d2.cattle/datasets/keypoints/coco_format/train_imgs
!ln -sf /home/ptthang/d2.cattle/Ground_truth_aligned/aligned_train.json /home/ptthang/d2.cattle/datasets/keypoints/coco_format/annotations/train.json